In [2]:
# Análise de sentimentos com base nos comentários do site vivino
# passos
# 1. Importar bibliotecas
# 2. Importar dados
# 3. Limpeza, tratamento e preparação dos dados (tradução de comentários para pt-br)
# 4. Classificação dos comentários com base no sentimento usando TextBlob
# 5. Realizar uma contagem de palavras para cada sentimento
# 6. Replicar o processo para o reddit

In [3]:
# 1. Importar bibliotecas
# !pip install googletrans==3.1.0a0 --force-reinstall
import re
import pandas as pd
import plotly.express as px
from googletrans import Translator
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# 1.1. Download de pacotes do NLTK
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\gugat\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gugat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gugat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# Criando função para os passos 3, 4 e 5 para replicar para os outros dataframes
def limpeza(df, col: str):
    #linhas duplicadas
    # print(df.duplicated().sum())
    # print(df.duplicated())
    df.drop_duplicates(subset=[col], inplace=True)
    # print(df.shape)

    #traduzir comentários para pt-br
    translator = Translator()
    df[col] = df[col].apply(lambda x: translator.translate(x, dest='en').text)

    #remover stopwords
    stop_words = set(stopwords.words())
    #adicionar palavras que não fazem sentido para o contexto
    # stop_words.update(
    #     [
    #         'product', 'purchase', 'I bought', 'buy', 'bought', 'we bought', 'they buy', 'and', 'in the',
    #         'the', 'a', 'some', 'of', 'for', 'by', 'with', 'without', 'about', 'under', 'between', 'until'
    #      ]
    # )
    df[col] = df[col].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

    #remover pontuação
    df[col] = df[col].str.replace('[^\w\s]','')

    #remover espaços em branco
    df[col] = df[col].str.strip()

    return df

def classificacao(texto, col = None):
    # Se o texto for um dataframe
    if type(texto) == pd.core.frame.DataFrame:
        # Classificação dos comentários com base no sentimento usando TextBlob
        # Classificar comentários onde se for maior que 0.0 é positivo, menor que 0.0 é negativo e igual a 0.0 é neutro
        # texto['Sentimento'] = texto[col].apply(lambda x: TextBlob(x).sentiment.polarity)
        # texto['Sentimento'] = texto['Sentimento'].apply(lambda x: 'positivo' if x > 0.0 else ('negativo' if x < 0.0 else 'neutro'))

        # Agrupar comentários por sentimento
        df_sentimento = texto.groupby('Sentimento').count().reset_index()

        # Visualizar quantidade de comentários por sentimento usando gráfico de barras onde neutro é azul, positivo é verde e negativo é vermelho
        fig = px.bar(df_sentimento, x='Sentimento', y='Avaliação', color='Sentimento', color_discrete_map={'positivo':'green', 'negativo':'red', 'neutro':'blue'})
        fig.show()
    else:
        pass

    # Realizar uma contagem de palavras para cada sentimento
    # Criar uma lista de palavras para cada sentimento
    positivo = []
    negativo = []
    neutro = []

    # Se o texto for um dataframe
    if type(texto) == pd.core.frame.DataFrame:
        # Adicionar palavras a lista de acordo com o sentimento
        for i in texto[texto['Sentimento'] == 'positivo'][col]:
            positivo.append(i)
        for i in texto[texto['Sentimento'] == 'negativo'][col]:
            negativo.append(i)
        for i in texto[texto['Sentimento'] == 'neutro'][col]:
            neutro.append(i)
    # Se o texto for uma lista
    elif type(texto) == list:
        for i, review in enumerate(texto):
            try:
                sentiment = sentiments[i]
            except:
                sentiment = TextBlob(review).sentiment.polarity
                sentiment = 'positive' if sentiment > 0.0 else ('negative' if sentiment < 0.0 else 'neutral')
            # print(sentiment)
            sentiment_map = {
                "positive": positivo,
                "negative": negativo,
                "neutral": neutro
            }
            sentiment_map[sentiment].append(review)
    else:
        pass

    # Criar uma lista de palavras para cada sentimento
    positivo = ' '.join(positivo)
    negativo = ' '.join(negativo)
    neutro = ' '.join(neutro)

    # Criar uma lista de palavras para cada sentimento
    positivo = TextBlob(positivo)
    negativo = TextBlob(negativo)
    neutro = TextBlob(neutro)

    # Criar uma lista de palavras para cada sentimento
    positivo = positivo.word_counts.items()
    negativo = negativo.word_counts.items()
    neutro = neutro.word_counts.items()

    # Criar um dataframe para cada sentimento
    df_positivo = pd.DataFrame(positivo, columns=['Palavra', 'Contagem'])
    df_negativo = pd.DataFrame(negativo, columns=['Palavra', 'Contagem'])
    df_neutro = pd.DataFrame(neutro, columns=['Palavra', 'Contagem'])

    # Se o texto for um dataframe
    if type(texto) == pd.core.frame.DataFrame:
        return texto, df_positivo, df_negativo, df_neutro
    # Se o texto for uma lista
    else:
        return df_positivo, df_negativo, df_neutro

def visualizacao(df_positivo, df_negativo, df_neutro):
    # Visualização dos resultados
    # Visualizar quantidade de comentários por sentimento usando gráfico de barras
    # Visualizar as 10 palavras mais frequentes para cada sentimento
    # Visualizar as 10 palavras mais frequentes para o sentimento positivo
    df_positivo = df_positivo.sort_values(by='Contagem', ascending=False)
    fig = px.bar(df_positivo.head(10), x='Palavra', y='Contagem', color='Palavra', color_discrete_map={'Palavra':'green'}, title= 'Positivo')
    fig.show()

    # Visualizar as 10 palavras mais frequentes para o sentimento negativo
    df_negativo = df_negativo.sort_values(by='Contagem', ascending=False)
    fig = px.bar(df_negativo.head(10), x='Palavra', y='Contagem', color='Palavra', color_discrete_map={'Palavra':'red'}, title= 'Negativo')
    fig.show()

    # Visualizar as 10 palavras mais frequentes para o sentimento neutro
    df_neutro = df_neutro.sort_values(by='Contagem', ascending=False)
    fig = px.bar(df_neutro.head(10), x='Palavra', y='Contagem', color='Palavra', color_discrete_map={'Palavra':'blue'}, title= 'Neutro')
    fig.show()

    return


In [5]:
# 2. Importar dados vivino
df_vivino1 = pd.read_excel(r'../data/extracao_vivino_pinot_noir.xlsx')
df_vivino2 = pd.read_excel('../data/extracao_vivino_malbec.xlsx')
df_vivino3 = pd.read_excel('../data/extracao_vivino_chardonnay.xlsx')

In [6]:
# 2.1. visualizar as colunas dos dataframes
print(df_vivino1.columns)
print(df_vivino2.columns)
print(df_vivino3.columns)

Index(['Nome Vinho', 'Localidade', 'Harmoniza', 'Avaliação'], dtype='object')
Index(['Nome Vinho', 'Localidade', 'Harmoniza', 'Avaliação'], dtype='object')
Index(['Nome Vinho', 'Localidade', 'Harmoniza', 'Avaliação'], dtype='object')


In [7]:
# 2.2. Mesclar os dataframes
# 2.2.1. Colocar uma coluna com a respectiva uva antes de mesclar
df_vivino1['Uva'] = 'Pinot Noir'
df_vivino2['Uva'] = 'Malbec'
df_vivino3['Uva'] = 'Chardonnay'

# 2.2.2. Mesclar os dataframes
df_vivino = pd.concat([df_vivino1, df_vivino2, df_vivino3])
df_vivino

,Nome Vinho,Localidade,Harmoniza,Avaliação,Uva
0,Anderson Hill O Series Pinot Noir 2018,"Adelaide Hills, Austrália","Carne de vaca, Vitela, Carne de caça (cervo, v...",Excelente pinot noir da Austrália. Medium body...,Pinot Noir
1,Anderson Hill O Series Pinot Noir 2018,"Adelaide Hills, Austrália","Carne de vaca, Vitela, Carne de caça (cervo, v...","On the nose, red plum, cherry cola, red cherry...",Pinot Noir
2,Anderson Hill O Series Pinot Noir 2018,"Adelaide Hills, Austrália","Carne de vaca, Vitela, Carne de caça (cervo, v...","Strawberry compote, cherry ripe and shitake mu...",Pinot Noir
3,Anderson Hill O Series Pinot Noir 2018,"Adelaide Hills, Austrália","Carne de vaca, Vitela, Carne de caça (cervo, v...",Vintage: 2017 Taste & Nose: Aged tangerine peel,Pinot Noir
4,Anderson Hill O Series Pinot Noir 2018,"Adelaide Hills, Austrália","Carne de vaca, Vitela, Carne de caça (cervo, v...",Safra 2021,Pinot Noir
...,...,...,...,...,...
486,William Fèvre Chile La Misiōn Chardonnay Reser...,"Pirque, Chile","Porco, Peixe gordo (salmão, atum, etc.), Veget...",good,Chardonnay
487,William Fèvre Chile La Misiōn Chardonnay Reser...,"Pirque, Chile","Porco, Peixe gordo (salmão, atum, etc.), Veget...",Water mineral,Chardonnay
488,William Fèvre Chile La Misiōn Chardonnay Reser...,"Pirque, Chile","Porco, Peixe gordo (salmão, atum, etc.), Veget...","Green pear, grape and citrus. A simple and fla...",Chardonnay
489,William Fèvre Chile La Misiōn Chardonnay Reser...,"Pirque, Chile","Porco, Peixe gordo (salmão, atum, etc.), Veget...",Safra 2020,Chardonnay


In [8]:
# 3. Limpeza, tratamento e preparação dos dados (tradução de comentários para pt-br)
dfx = limpeza(df_vivino, 'Avaliação')

C:\Users\gugat\AppData\Local\Temp\ipykernel_30128\3786326699.py:25: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col] = df[col].str.replace('[^\w\s]','')


In [9]:
# Classifica os reviews
from sentiment_analysis import ReviewClassifier


classifier = ReviewClassifier()
# dfx["Sentimento"] = dfx["Avaliação"].apply(lambda review: classifier.get_sentiment(review))
# dfx

In [10]:
# dfx.to_csv("df_classificado.csv")
dfx = pd.read_csv("../data/df_classificado.csv")
dfx.drop(columns=['Unnamed: 0'], inplace=True)
dfx.dropna(inplace=True)
dfx

,Nome Vinho,Localidade,Harmoniza,Avaliação,Uva,Sentimento
0,Anderson Hill O Series Pinot Noir 2018,"Adelaide Hills, Austrália","Carne de vaca, Vitela, Carne de caça (cervo, v...",Excellent Pinot Noir Australia Medium body med...,Pinot Noir,positive
1,Anderson Hill O Series Pinot Noir 2018,"Adelaide Hills, Austrália","Carne de vaca, Vitela, Carne de caça (cervo, v...",On nose red plum cherry cola red cherry clove ...,Pinot Noir,neutral
2,Anderson Hill O Series Pinot Noir 2018,"Adelaide Hills, Austrália","Carne de vaca, Vitela, Carne de caça (cervo, v...",Strawberry compote cherry ripe shitake mushroo...,Pinot Noir,neutral
3,Anderson Hill O Series Pinot Noir 2018,"Adelaide Hills, Austrália","Carne de vaca, Vitela, Carne de caça (cervo, v...",Vintage 2017 Taste Nose Aged tangerine peel,Pinot Noir,neutral
4,Anderson Hill O Series Pinot Noir 2018,"Adelaide Hills, Austrália","Carne de vaca, Vitela, Carne de caça (cervo, v...",Harvest 2021,Pinot Noir,neutral
...,...,...,...,...,...,...
1572,William Fèvre Chile La Misiōn Chardonnay Reser...,"Pirque, Chile","Porco, Peixe gordo (salmão, atum, etc.), Veget...",Citrus French Chardonnay nearby supermarket,Chardonnay,neutral
1573,William Fèvre Chile La Misiōn Chardonnay Reser...,"Pirque, Chile","Porco, Peixe gordo (salmão, atum, etc.), Veget...",Yasuuma Chardonnay,Chardonnay,neutral
1574,William Fèvre Chile La Misiōn Chardonnay Reser...,"Pirque, Chile","Porco, Peixe gordo (salmão, atum, etc.), Veget...",Mineral crisp ok buy again,Chardonnay,neutral
1576,William Fèvre Chile La Misiōn Chardonnay Reser...,"Pirque, Chile","Porco, Peixe gordo (salmão, atum, etc.), Veget...",Water mineral,Chardonnay,neutral


In [11]:
# 3.1. Achar as avaliações que só estão escritos "Safra 2018" por exemplo mas achar para todos os anos usando regex para dropar a linha
dfx1 = dfx[~dfx['Avaliação'].str.match(r'^Safra \d{4}$')]
dfx1["Sentimento"] = dfx1["Sentimento"].replace({"positive": "positivo", "negative": "negativo", "neutral": "neutro"})
dfx1

,Nome Vinho,Localidade,Harmoniza,Avaliação,Uva,Sentimento
0,Anderson Hill O Series Pinot Noir 2018,"Adelaide Hills, Austrália","Carne de vaca, Vitela, Carne de caça (cervo, v...",Excellent Pinot Noir Australia Medium body med...,Pinot Noir,positivo
1,Anderson Hill O Series Pinot Noir 2018,"Adelaide Hills, Austrália","Carne de vaca, Vitela, Carne de caça (cervo, v...",On nose red plum cherry cola red cherry clove ...,Pinot Noir,neutro
2,Anderson Hill O Series Pinot Noir 2018,"Adelaide Hills, Austrália","Carne de vaca, Vitela, Carne de caça (cervo, v...",Strawberry compote cherry ripe shitake mushroo...,Pinot Noir,neutro
3,Anderson Hill O Series Pinot Noir 2018,"Adelaide Hills, Austrália","Carne de vaca, Vitela, Carne de caça (cervo, v...",Vintage 2017 Taste Nose Aged tangerine peel,Pinot Noir,neutro
4,Anderson Hill O Series Pinot Noir 2018,"Adelaide Hills, Austrália","Carne de vaca, Vitela, Carne de caça (cervo, v...",Harvest 2021,Pinot Noir,neutro
...,...,...,...,...,...,...
1572,William Fèvre Chile La Misiōn Chardonnay Reser...,"Pirque, Chile","Porco, Peixe gordo (salmão, atum, etc.), Veget...",Citrus French Chardonnay nearby supermarket,Chardonnay,neutro
1573,William Fèvre Chile La Misiōn Chardonnay Reser...,"Pirque, Chile","Porco, Peixe gordo (salmão, atum, etc.), Veget...",Yasuuma Chardonnay,Chardonnay,neutro
1574,William Fèvre Chile La Misiōn Chardonnay Reser...,"Pirque, Chile","Porco, Peixe gordo (salmão, atum, etc.), Veget...",Mineral crisp ok buy again,Chardonnay,neutro
1576,William Fèvre Chile La Misiōn Chardonnay Reser...,"Pirque, Chile","Porco, Peixe gordo (salmão, atum, etc.), Veget...",Water mineral,Chardonnay,neutro


In [12]:
# 4. Classificação dos comentários com base no sentimento usando TextBlob
dfx2, df_positivo, df_negativo, df_neutro = classificacao(dfx1, 'Avaliação')

In [13]:
# 5. Visualização dos resultados
visualizacao(df_positivo, df_negativo, df_neutro)

In [14]:
dfx1.query("Sentimento == 'negativo'")['Avaliação'].to_list()

['Not',
 'A massacre',
 'Bloody creepy bitch A complete experience begins aromatic explosion high complexity great freshness An incredible mouth integrated acidity flavors medium body prolonged finish',
 'Nothing add nobility Malbec Just acidic aftertaste Quite aromatic Wine abusive price BR Its worth buying winery']

In [15]:
# 6. Replicar para o reddit
with open("../data/reddit.txt", 'r', encoding='utf-8') as file:
    texto = file.read()
    # Tokeniza as palavras no texto
    tokens = word_tokenize(texto)
palavras = [word.lower() for word in tokens]
# Remove as stopwords da lista de palavras
stopwords_list = set(stopwords.words('portuguese'))  # Altere para o idioma desejado, se necessário
#adicionar palavras que não fazem sentido para o contexto
stopwords_list.update(['produto', 'compra', 'comprei', 'comprar', 'comprado', 'comprada', 'compramos', 'compram', 'and',
                    'na', 'no', 'nas', 'nos', 'o', 'a', 'os', 'as', 'um', 'uma', 'uns', 'umas', 'de', 'do', 'da', 'lol',
                    'dos', 'das', 'em', 'para', 'por', 'com', 'sem', 'sobre', 'sob', 'entre', 'até', 'ate', 'até','haha', 'ento'
                    ])
palavras = [word for word in palavras if word not in stopwords_list]
palavras = [re.sub(r'[^a-zA-Z0-9]', '', word) for word in palavras if word not in stopwords_list]

# Remove palavras vazias
palavras = [word for word in palavras if word]

In [16]:
len(palavras)

216547

In [17]:
# Classifica as palavras em positivas, negativas e neutras e depois conta a frequência para cada classe
df_positivo2, df_negativo2, df_neutro2 = classificacao(palavras)

In [19]:
# Visualização dos resultados
visualizacao(df_positivo2, df_negativo2, df_neutro2)

In [26]:
df_negativo.Palavra.to_list()

['not',
 'a',
 'massacre',
 'bloody',
 'creepy',
 'bitch',
 'complete',
 'experience',
 'begins',
 'aromatic',
 'explosion',
 'high',
 'complexity',
 'great',
 'freshness',
 'an',
 'incredible',
 'mouth',
 'integrated',
 'acidity',
 'flavors',
 'medium',
 'body',
 'prolonged',
 'finish',
 'nothing',
 'add',
 'nobility',
 'malbec',
 'just',
 'acidic',
 'aftertaste',
 'quite',
 'wine',
 'abusive',
 'price',
 'br',
 'its',
 'worth',
 'buying',
 'winery']

In [37]:
# Separando palavras exclusivamente negativas
negative_words = set(df_negativo2.Palavra.to_list())
positive_words = set(df_positivo2.Palavra.to_list())
neutral_words = set(df_neutro2.Palavra.to_list())

negative_words = negative_words - positive_words - neutral_words
list(negative_words)

['lame',
 'extreme',
 'dry',
 'wtf',
 'little',
 'limited',
 'duh',
 'heavy',
 'bad',
 'center',
 'sentimental',
 'aged',
 'nameless',
 'half',
 'sec',
 'grey',
 'broken',
 'drunk',
 'casual',
 'alas',
 'base',
 'brutal',
 'black',
 'small',
 'twisted',
 'sinister',
 'shy',
 'slight',
 'wet',
 'long',
 'coma',
 'ruins',
 'cold',
 'shit',
 'random',
 'negative',
 'flat',
 'slow',
 'swill',
 'late',
 'secret',
 'blind',
 'single',
 'artificial',
 'chicken',
 'due',
 'limp',
 'unknown',
 'down',
 'polar',
 'pink',
 'sophomoric',
 'tame',
 'green',
 'mental',
 'sour',
 'endless',
 'monkey',
 'criminal',
 'round',
 'humble',
 'vacuum',
 'everyday',
 'usual',
 'rude',
 'cruel',
 'propaganda',
 'unilateral',
 'complex',
 'crazy']

In [39]:
df_positivo2.Palavra.to_list()

['top',
 'grand',
 'particular',
 'rich',
 'noble',
 'rose',
 'normal',
 'super',
 'light',
 'special',
 'more',
 'high',
 'familiar',
 'ok',
 'free',
 'natural',
 'cool',
 'amazing',
 'real',
 'popular',
 'legal',
 'superior',
 'precise',
 'best',
 'new',
 'crisp',
 'old',
 'joy',
 'fantastic',
 'genuine',
 'ideal',
 'far',
 'sexual',
 'sexy',
 'original',
 'innocent',
 'strong',
 'young',
 'pure',
 'lol',
 'delicious',
 'great',
 'motley',
 'wow',
 'lmao',
 'vital',
 'first',
 'advanced',
 'win',
 'haha',
 'good',
 'love',
 'wonderful',
 'rare',
 'sweet',
 'direct',
 'full',
 'loving',
 'cheap',
 'smile',
 'much',
 'magic',
 'awesome',
 'classic',
 'aww',
 'apt',
 'cultural',
 'social',
 'near',
 'true',
 'experimental',
 'action',
 'nice',
 'pleasant',
 'soft',
 'older',
 'favorite',
 'hot',
 'brave',
 'happy',
 'hahaha',
 'thanks',
 'winning',
 'fresh',
 'fun',
 'wild',
 'enjoy',
 'trendy',
 'whole',
 'bonny',
 'golden',
 'fine',
 'most',
 'fast']